In [2]:
from biorange.ppi import ppi_final
import pandas as pd

In [5]:
#预处理kegg数据,将基因名展开
import pandas as pd
# 读取数据
kegg_df = pd.read_csv('/home/liuyan/projects/package/biorange/notebooks/kegg_test.csv')

# 展开'gene'列
kegg_df['gene_name'] = kegg_df['gene_name'].str.split(';')
kegg_df_exploded = kegg_df.explode('gene_name')

# 保存结果
# df_exploded.to_csv('expanded_kegg_file.csv', index=False)
kegg_gene=kegg_df_exploded[['gene_name','term']]
print(kegg_gene)

   gene_name                                             term
0     CDKN1A  Kaposi sarcoma-associated herpesvirus infection
0      IFNB1  Kaposi sarcoma-associated herpesvirus infection
0        FOS  Kaposi sarcoma-associated herpesvirus infection
0      PTGS2  Kaposi sarcoma-associated herpesvirus infection
0       FGF2  Kaposi sarcoma-associated herpesvirus infection
..       ...                                              ...
14       LEP                  Adipocytokine signaling pathway
14       NPY                  Adipocytokine signaling pathway
14      LEPR                  Adipocytokine signaling pathway
14       TNF                  Adipocytokine signaling pathway
14      RELA                  Adipocytokine signaling pathway

[156 rows x 2 columns]


In [8]:
def get_ppi_relationships(gene_names, species_id=9606):
    data = ppi_final.fetch_ppi_data(gene_names, species_id)
    if data is None:
        return None

    nodes = ppi_final.parse_interaction_data(data)
    if nodes is None:
        return None

    return nodes
gene_interaction=get_ppi_relationships(kegg_gene['gene_name'])
print(gene_interaction)

    preferredName_A preferredName_B
0               IL2            NOS3
1               IL2           PTPN2
2               IL2           IKBKB
3               IL2           CREB1
4               IL2            BDNF
..              ...             ...
287            IL10           IKBKB
288            IL10            BDNF
289            IL10           HIF1A
290            BDNF           HIF1A
291           IKBKB           HIF1A

[292 rows x 2 columns]


In [ ]:
# 寻找kegg_gene中['gene']对应的成分名
#将三个数据库的结果收集起来，合并为一个数据，再和含有成分名表进行合并
kegg_ingredient = []

In [ ]:
import pandas as pd

def generate_files(kegg_gene, kegg_ingredient, gene_interaction):
    # 生成 node_file
    node_file_df = pd.concat([
        kegg_gene[['gene', 'term']].rename(columns={'gene': 'node1', 'term': 'node2'}),
        kegg_ingredient[['gene', 'ingredient_name']].rename(columns={'gene': 'node1', 'ingredient_name': 'node2'}),
        gene_interaction[['preferredName_A', 'preferredName_B']].rename(columns={'preferredName_A': 'node1', 'preferredName_B': 'node2'}),
    ], ignore_index=True)
    print(node_file_df)

    # 生成 type_file
    type_file_df = pd.concat([
        kegg_gene[['term']].rename(columns={'term': 'node'}),
        kegg_gene[['gene']].rename(columns={'gene': 'node'}),
        kegg_ingredient[['ingredient_name']].rename(columns={'ingredient_name': 'node'}),
    ])

    type_file_df['type'] = ['pathway'] * len(kegg_gene) + ['ingredient_name'] * len(kegg_ingredient) + ['gene'] * len(kegg_gene)
    type_file_df = type_file_df.drop_duplicates()
    print(type_file_df)

    return node_file_df, type_file_df

# 调用函数并传递数据
node_file_df, type_file_df = generate_files(kegg_gene, kegg_ingredient, gene_interaction)

In [1]:
#生成node和Type文件

# 生成node_file
node_file_df = pd.concat([
    kegg_gene[['gene','term']].rename(columns={'gene': 'node1', 'term': 'node2'}),
    kegg_ingredient[['gene', 'ingredient_name']].rename(columns={'gene': 'node1', 'ingredient_name': 'node2'}),
    gene_interaction[['preferredName_A','preferredName_B']].rename(columns={'preferredName_A': 'node1', 'preferredName_B': 'node2'}),
], ignore_index=True)
print(node_file_df)

#生成type_file
type_file_df = pd.concat([
    kegg_gene[['term']].rename(columns={'term': 'node'}),
    kegg_gene[['gene']].rename(columns={'gene': 'node'}),
    kegg_ingredient[['ingredient_name']].rename(columns={'ingredient_name': 'node'}),
])

type_file_df['type'] = ['pathway'] * len(kegg_gene) + ['ingredient_name'] * len(kegg_ingredient) + ['gene'] * len(kegg_gene)
type_file_df = type_file_df.drop_duplicates()
print(type_file_df)
# 输出结果到文件
node_file_df.to_csv('node_file.txt', sep='\t', index=False)
type_file_df.to_csv('type_file.txt', sep='\t', index=False)

NameError: name 'pd' is not defined

In [24]:
 # 输入示例
gene_names_file = "/home/liuyan/projects/package/biorange/biorange/data/shared targets.csv"
gene_names = pd.read_csv(gene_names_file)["shared_targets"]
# output_dir = "./"
# 只获取ppi节点关系表
gg = get_ppi_relationships(gene_names)
print(gg)

     preferredName_A preferredName_B
0               CFTR           BRCA1
1               CFTR            EGFR
2               CFTR            KRAS
3               CFTR            TP53
4               CFTR           CHEK2
...              ...             ...
1643             MYC          HNRNPU
1644             MYC          NOTCH1
1645             MYC          CTNNB1
1646           RAD51           MUTYH
1647          CTNNB1          NOTCH1

[1648 rows x 2 columns]
